In [13]:
import SimpleITK as sitk
import os
import shutil
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import pydicom


# Standardize Spacing and Resolution 


In [41]:

# Load the NIfTI image using SimpleITK
image = sitk.ReadImage('/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/training/patient098/patient098_frame01.nii/42Gate0.nii')

# Get the current spacing and size
current_spacing = np.array(image.GetSpacing())  # Current spacing (z, y, x)
current_size = np.array(image.GetSize())  # Current size (x, y, z)

# Define the target spacing
target_spacing = np.array([1.0, 1.0, 1.0])

# Calculate the resize factor
resize_factor = current_spacing / target_spacing

# Calculate new size (real-world dimensions) based on resize factor
new_real_shape = current_size * resize_factor
new_shape = np.round(new_real_shape).astype(int)

# Convert the SimpleITK image to a NumPy array
image_np = sitk.GetArrayFromImage(image)  # Get NumPy array (z, y, x)

# Apply zoom with the resize factor, note we reverse the zoom factors to match NumPy's (z, y, x) order
image_resampled_np = zoom(image_np, resize_factor[::-1], order=3)  # cubic interpolation

# Convert the resampled NumPy array back to SimpleITK image
image_resampled = sitk.GetImageFromArray(image_resampled_np)

# Update the spacing to reflect the new voxel size
image_resampled.SetSpacing(target_spacing)

# Print the new shape and spacing
print(f"Original shape: {image_np.shape}, Resampled shape: {image_resampled_np.shape}")
print(f"Original spacing: {current_spacing}, New spacing: {image_resampled.GetSpacing()}")

# Optionally, save the resampled image
sitk.WriteImage(image_resampled, 'resampled_image.nii')


NiftiImageIO (0x1526b7230): /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/training/patient098/patient098_frame01.nii/42Gate0.nii has unexpected scales in sform

NiftiImageIO (0x1526b7230): /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/training/patient098/patient098_frame01.nii/42Gate0.nii has unexpected scales in sform



Original shape: (7, 256, 216), Resampled shape: (70, 320, 270)
Original spacing: [ 1.25  1.25 10.  ], New spacing: (1.0, 1.0, 1.0)


In [ ]:

# Standardize the spacing

def clone_directory_structure(src, dst):
    """
    Clone the directory structure from src to dst.
    """
    for root, dirs, files in os.walk(src):
        # Create the corresponding directory in the destination
        dst_dir = root.replace(src, dst, 1)  # Replace src with dst in the path
        os.makedirs(dst_dir, exist_ok=True)  # Create the directory if it doesn't exist

def process_single_slice_dicom(dicom_file, target_resolution, output_dir):
    """
    Load and standardize the spacing of a single-slice DICOM file.
    """
    
    # Load the DICOM file
    image = sitk.ReadImage(dicom_file)
    current_spacing = np.array(image.GetSpacing())

    resize_factor = np.array([current_spacing[0] / target_resolution[0], 
                               current_spacing[1] / target_resolution[1], 
                               1.0])

    new_real_shape = np.array(image.GetSize()) * resize_factor

    new_shape = np.round(new_real_shape).astype(int)
    
    real_resize_factor = new_shape / np.array(image.GetSize())
    
    image_np = sitk.GetArrayFromImage(image)

    image_resampled_np = zoom(image_np, real_resize_factor[::-1], order=1)

    image_resampled = sitk.GetImageFromArray(image_resampled_np)

    new_spacing = np.array([target_resolution[0], target_resolution[1], current_spacing[2]])

    image_resampled.SetSpacing(new_spacing)
    
    output_file = os.path.join(output_dir, os.path.basename(dicom_file))
    sitk.WriteImage(image_resampled, output_file)

    print(f"Saved: {output_file}")
    print(f"Original shape: {image_np.shape}, Resampled shape: {image_resampled_np.shape}")
    print(f"Original spacing: {current_spacing}, New spacing: {image_resampled.GetSpacing()}")
    

def loop_and_standardize(dataset_path, target_resolution, output_path, search_term="4d"):
    """
    Loop over the dataset, clone the folder structure, and standardize the resolution of single-slice data.
    """
    # Clone the directory structure
    clone_directory_structure(dataset_path, output_path)

    for root, dirs, files in os.walk(dataset_path):
        # Ignore directories with '4d' in their name
        if search_term not in os.path.basename(root):
            for file in files:
                if search_term not in file:
                    if file.endswith(".nii"):  # Assuming DICOM files are used
                        dicom_file_path = os.path.join(root, file)
                        print(f"Processing: {dicom_file_path}")
                        
                        # Process and save in the corresponding output directory
                        output_dir = root.replace(dataset_path, output_path, 1)
                        process_single_slice_dicom(dicom_file_path, target_resolution, output_dir)

# Example usage:
dataset_path = "/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/training"  # Original dataset path
output_path = "/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized"  # New path for standardized data
target_spacing = np.array([1.0, 1.0,1.0])  # Example target resolution (x, y, z spacing)

loop_and_standardize(dataset_path, target_spacing, output_path)


In [48]:

def check_unified_resolution(output_path):
    """
    Check if all images in the output directory have unified resolution.
    """
    resolutions = set()  # To store unique resolutions

    for root, dirs, files in os.walk(output_path):
    
        for file in files:
            # print(file)
            if file.endswith(".nii"):  # Check for NIfTI files
                nifti_file_path = os.path.join(root, file)
                try:
                    image = sitk.ReadImage(nifti_file_path)
                    spacing = image.GetSpacing()  # Get the spacing (resolution)
                    resolutions.add(spacing)  # Add to the set of unique resolutions

                except Exception as e:
                    print(f"Failed to read {nifti_file_path}: {str(e)}")

    # Check the number of unique resolutions
    if len(resolutions) == 1:
        print("All images have a unified resolution.")
    else:
        print("There are multiple resolutions in the dataset:")
        for res in resolutions:
            print(res)

# Example usage:
output_path = "/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized"  # Path where standardized images are saved
check_unified_resolution(output_path)


There are multiple resolutions in the dataset:
(1.0, 1.0, 10.0)
(1.0, 1.0, 5.0)
(1.0, 1.0, 6.5)
(1.0, 1.0, 7.0)


In [ ]:
# standardize the resolution 
def process_Resolution_by_padding(dicom_file, target_resolution, output_dir):
    """
    Load and standardize the resolution of a single-slice DICOM file using padding.
    """
    image = sitk.ReadImage(dicom_file)
    constant_val = int(sitk.GetArrayFromImage(image).min())

    current_size = np.array(image.GetSize())
    padding_left_right = target_resolution[0] - current_size[0]
    padding_top_bottom = target_resolution[1] - current_size[1]
    padding_left = int(padding_left_right // 2)
    padding_right = int(padding_left_right - padding_left)
    padding_top = int(padding_top_bottom // 2)
    padding_bottom = int(padding_top_bottom - padding_top)

    transformed = sitk.ConstantPad(image,(padding_left,padding_top,0),(padding_right,padding_bottom,0),constant_val)
    
    output_file = os.path.join(output_dir, os.path.basename(dicom_file))
    sitk.WriteImage(transformed, output_file)

def loop_and_standardize(dataset_path, target_resolution, output_path, search_term="4d"):
    """
    Loop over the dataset, clone the folder structure, and standardize the resolution of single-slice data.
    """
    # Clone the directory structure
    # clone_directory_structure(dataset_path, output_path)

    for root, dirs, files in os.walk(dataset_path):
        # Ignore directories with '4d' in their name
        if search_term not in os.path.basename(root):
            for file in files:
                if search_term not in file:
                    if file.endswith(".nii"):  # Assuming DICOM files are used
                        dicom_file_path = os.path.join(root, file)
                        print(f"Processing: {dicom_file_path}")
                        
                        # Process and save in the corresponding output directory
                        output_dir = root.replace(dataset_path, output_path, 1)
                        process_Resolution_by_padding(dicom_file_path, target_resolution, output_dir)

# Example usage:
dataset_path = "/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized"  # Original dataset path
output_path = "/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized"  # New path for standardized data
target_spacing = np.array([512, 512,1.0])  # Example target resolution (x, y, z spacing)

loop_and_standardize(dataset_path, target_spacing, output_path)

In [53]:
# check if their is a resolution more than 512x512
def check_resolution(output_path):
    """
    Check if all images in the output directory have a resolution of 512x512.
    """
    for root, dirs, files in os.walk(output_path):
        for file in files:
            if file.endswith(".nii"):  # Check for NIfTI files
                nifti_file_path = os.path.join(root, file)
                try:
                    image = sitk.ReadImage(nifti_file_path)
                    size = image.GetSize()  # Get the size (resolution)
                    if size[0] == 512 or size[1] == 512:
                        print(f"Resolution is 512x512 for: {nifti_file_path}")
                    else:
                        print(f"Resolution lower 512x512 for: {nifti_file_path}")
                except Exception as e:
                    print(f"Failed to read {nifti_file_path}: {str(e)}")

output_path = "/Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized"  # Path where standardized images are saved
check_resolution(output_path)

Resolution is 512x512 for: /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized/patient015/patient015_frame10_gt.nii/DCM27_V3_9.nii
Resolution is 512x512 for: /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized/patient015/patient015_frame01.nii/0.nii
Resolution is 512x512 for: /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized/patient015/patient015_frame10.nii/9.nii
Resolution is 512x512 for: /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized/patient015/patient015_frame01_gt.nii/DCM27_V3_0.nii
Resolution is 512x512 for: /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized/patient012/patient012_frame13.nii/23Gate12.nii
Resolution is 512x512 for: /Users/ahmed_ali/Documents/GitHub/GP-2025-Strain/Data/ACDC/database/train_standardized/patient012/patient012_frame01.nii/0.nii
Resolution is 512x512 for: /Users/ahmed_ali/D